In [ ]:
import keras
from keras.models import Model, Input
from keras.layers import Conv2D, Activation, MaxPool2D, Flatten, Dense
from keras.layers import BatchNormalization
from keras.datasets import mnist
from keras.optimizers import SGD

from keras.layers.advanced_activations import PReLU

from keras.utils import to_categorical

import numpy as np

from numpy.random import seed
seed(42)

import time

import matplotlib.pyplot as plt

# Magic numbers

In [ ]:
nb_classes = 10
optim = SGD()
batch_size = 128
epochs = 20
pix_ox, pix_oy = (28,28)

want_to_train = False

# Load the dataset

In [ ]:
print(mnist.__file__)

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Inspect the dataset

In [ ]:
print("Shape of x_train: ",x_train.shape)
print("Shape of y_train: ",y_train.shape)
print("Shape of x_test: ",x_test.shape)
print("Shape of y_test: ",y_test.shape)

# Reshape data to 4D tensors

In [ ]:
x_train = np.reshape(x_train, (-1,pix_ox, pix_oy,1))
x_test = np.reshape(x_test, (-1,pix_ox, pix_oy, 1))

print("Shape of x_train: ",x_train.shape)
print("Shape of x_test: ",x_test.shape)

# Ensure float

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize the data

In [ ]:
x_train = x_train / 255.
x_test = x_test / 255.

# One-hot encoding

In [ ]:
print(y_train[544])

In [ ]:
y_train_onehot = to_categorical(y_train, nb_classes)
y_test_onehot = to_categorical(y_test, nb_classes)

print(y_train_onehot[544])

# The Model

In [ ]:
def my_model(x):
    
    x = Conv2D(filters=32, kernel_size=(5,5), 
               strides=(1,1),padding= 'same')(x)
    x = PReLU()(x)
    x = MaxPool2D(pool_size=(2,2), strides=(2,2), padding='valid')(x)
    
    x = Conv2D(filters=64, kernel_size=(5,5), 
               strides=(1,1),padding= 'same')(x)
    x = PReLU()(x)
    x = MaxPool2D(pool_size=(2,2), strides=(1,1), padding='valid')(x)
    
    x = Conv2D(filters=128, kernel_size=(3,3), 
               strides=(1,1),padding= 'same')(x)
    x = PReLU()(x)
    x = MaxPool2D(pool_size=(2,2), strides=(1,1), padding='valid')(x)
    
    x = Flatten()(x)
    x = Dense(50)(x)
    x = PReLU()(x)
    x = Dense(2)(x)
    x = PReLU(name='ghita')(x)
    
    x = Dense(nb_classes, activation='softmax')(x)
    
    return x
      

# Functional API

In [ ]:
inputs = Input((pix_ox, pix_oy,1))
out = my_model(inputs)

model = Model(inputs=inputs, outputs=out)

model.summary()

# Compile 

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=optim, 
             metrics=['accuracy'])

# Callbacks

In [ ]:
class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.logs = []
        self.losses = []
        self.test_losses = []
        
        self.plot = plt.figure()
        
    def on_epoch_end(self, epoch, logs={}):
        self.x.append(self.i)
        self.logs.append(logs)
        self.losses.append(logs.get('loss'))
        self.test_losses.append(logs.get('val_loss'))
        
        plt.plot(self.x, self.losses, label='train')
        plt.plot(self.x, self.test_losses, label='test')
        plt.show()
        
        self.i += 1
losses_train = PlotLosses()

# Train

In [ ]:
if(want_to_train):
    start_time = time.time()
    model.fit(x_train, y_train_onehot,
             epochs=epochs,
             batch_size=batch_size,
             validation_data=(x_test, y_test_onehot),
             callbacks=[losses_train],
             verbose=1)
    end_time = time.time() - start_time

    print("Training time: ",end_time)
else:
    from keras.models import load_model

    model = load_model("model_conv.h5")



# Print test accuracy

In [ ]:
score = model.evaluate(x_test, y_test_onehot)

In [ ]:
print("accuracy: ",score[1])

In [ ]:
def plot_features(feats, y_test):
    c = ['#ff0000', '#ffff00', '#00ff00', '#00ffff', '#0000ff',
         '#ff00ff', '#990000', '#999900', '#009900', '#009999']
    
    for i in range(10):
        plt.plot(feats[y_test == i,0], 
                 feats[y_test == i,1], 
                 '.',c=c[i])
        
    plt.legend(['0','1','2','3','4','5','6','7','8','9'])
    plt.show()

In [ ]:
model_reduced = Model(model.input, 
                      outputs=model.get_layer('ghita').output)

In [ ]:
features = model_reduced.predict(x_test)

In [ ]:
plot_features(features, y_test)